## Lendo os dados

In [0]:
# Caminho do arquivo
file_path = "/FileStore/tables/podbank/bureau_balance.csv"

df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(file_path)

df.show()


+------------+--------------+------+
|SK_ID_BUREAU|MONTHS_BALANCE|STATUS|
+------------+--------------+------+
|     5715448|             0|     C|
|     5715448|            -1|     C|
|     5715448|            -2|     C|
|     5715448|            -3|     C|
|     5715448|            -4|     C|
|     5715448|            -5|     C|
|     5715448|            -6|     C|
|     5715448|            -7|     C|
|     5715448|            -8|     C|
|     5715448|            -9|     0|
|     5715448|           -10|     0|
|     5715448|           -11|     X|
|     5715448|           -12|     X|
|     5715448|           -13|     X|
|     5715448|           -14|     0|
|     5715448|           -15|     0|
|     5715448|           -16|     0|
|     5715448|           -17|     0|
|     5715448|           -18|     0|
|     5715448|           -19|     0|
+------------+--------------+------+
only showing top 20 rows



## Criando variáveis de flag temporal

In [0]:
df.createOrReplaceTempView("dados")

df_temp_01 = spark.sql("""
SELECT
    *,
      CASE
        WHEN MONTHS_BALANCE >= -3 THEN 1
        ELSE 0
    END AS U3M,
    CASE
        WHEN MONTHS_BALANCE >= -6 THEN 1
        ELSE 0
    END AS U6M,
    CASE
        WHEN MONTHS_BALANCE >= -12 THEN 1
        ELSE 0
    END AS U12M,
    CASE
        WHEN MONTHS_BALANCE >= -24 THEN 1
        ELSE 0
    END AS U24M
FROM dados
ORDER BY `SK_ID_BUREAU`;
""")
df_temp_01.createOrReplaceTempView("df_temp_01")
df_temp_01.show(5)

+------------+--------------+------+---+---+----+----+
|SK_ID_BUREAU|MONTHS_BALANCE|STATUS|U3M|U6M|U12M|U24M|
+------------+--------------+------+---+---+----+----+
|     5001709|             0|     C|  1|  1|   1|   1|
|     5001709|            -5|     C|  0|  1|   1|   1|
|     5001709|            -1|     C|  1|  1|   1|   1|
|     5001709|            -2|     C|  1|  1|   1|   1|
|     5001709|            -3|     C|  1|  1|   1|   1|
+------------+--------------+------+---+---+----+----+
only showing top 5 rows



In [0]:
df_temp_01.createOrReplaceTempView("df_temp_01")

count_status = spark.sql("""
SELECT COUNT(DISTINCT STATUS) AS distinct_count, STATUS
FROM df_temp_01
GROUP BY STATUS
""")
count_status.show(10)

+--------------+------+
|distinct_count|STATUS|
+--------------+------+
|             1|     5|
|             1|     1|
|             1|     0|
|             1|     4|
|             1|     3|
|             1|     2|
|             1|     C|
|             1|     X|
+--------------+------+



In [0]:
df_temp_01.createOrReplaceTempView("df_temp_01")

df_temp_02 = spark.sql("""
SELECT
    *,
      CASE
        WHEN STATUS = "C" THEN 1
        ELSE 0
    END AS STATUS_C,
    CASE
        WHEN STATUS = "3" THEN 1
        ELSE 0
    END AS STATUS_3,
    CASE
        WHEN STATUS = "0" THEN 1
        ELSE 0
    END AS STATUS_0,
    CASE
        WHEN STATUS = "2" THEN 1
        ELSE 0
    END AS STATUS_2,
    CASE
        WHEN STATUS = "X" THEN 1
        ELSE 0
    END AS STATUS_X,
    CASE
        WHEN STATUS = "5" THEN 1
        ELSE 0
    END AS STATUS_5,
    CASE
        WHEN STATUS = "1" THEN 1
        ELSE 0
    END AS STATUS_1
FROM df_temp_01
ORDER BY `SK_ID_BUREAU`;
""")
df_temp_02.createOrReplaceTempView("df_temp_01")
df_temp_02.show(5)

+------------+--------------+------+---+---+----+----+--------+--------+--------+--------+--------+--------+--------+
|SK_ID_BUREAU|MONTHS_BALANCE|STATUS|U3M|U6M|U12M|U24M|STATUS_C|STATUS_3|STATUS_0|STATUS_2|STATUS_X|STATUS_5|STATUS_1|
+------------+--------------+------+---+---+----+----+--------+--------+--------+--------+--------+--------+--------+
|     5001709|             0|     C|  1|  1|   1|   1|       1|       0|       0|       0|       0|       0|       0|
|     5001709|            -5|     C|  0|  1|   1|   1|       1|       0|       0|       0|       0|       0|       0|
|     5001709|            -1|     C|  1|  1|   1|   1|       1|       0|       0|       0|       0|       0|       0|
|     5001709|            -2|     C|  1|  1|   1|   1|       1|       0|       0|       0|       0|       0|       0|
|     5001709|            -3|     C|  1|  1|   1|   1|       1|       0|       0|       0|       0|       0|       0|
+------------+--------------+------+---+---+----+----+--

## Criando variáveis de primeira camada

In [0]:
from pyspark.sql.functions import col, round, sum, avg, max, min, when, count, lit

colunas_agregacao_total = ['STATUS_C','STATUS_3','STATUS_0','STATUS_2','STATUS_X','STATUS_5','STATUS_1']

colunas_flags = ['U3M','U6M', 'U12M', 'U24M']
expressoes_agregacao = []

for flag in colunas_flags:
  for coluna in colunas_agregacao_total:
    expressoes_agregacao.append(round(count(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_TT_{coluna.upper()}_{flag.upper()}_BUREAU_BL"))
    expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna)).otherwise(lit(None))), 2).alias(f"QT_MED_{coluna.upper()}_{flag.upper()}_BUREAU_BL"))
    expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MAX_{coluna.upper()}_{flag.upper()}_BUREAU_BL"))
    expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MIN_{coluna.upper()}_{flag.upper()}_BUREAU_BL"))


expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicar as expressões de agregação
df_temp_03 = df_temp_02.groupBy("SK_ID_BUREAU").agg(*expressoes_agregacao).orderBy("SK_ID_BUREAU")


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_03.columns
nomes_cols_novas = nomes_cols[1:]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_03.columns) - 1)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_03.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_03.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

# Mostrando o novo DataFrame com as variáveis criadas.
df_temp_03.show(5, False)

Quantidade Total de Variáveis Criadas: 112
Nomes das Variáveis Criadas: ['QT_TT_STATUS_C_U3M_BUREAU_BL', 'QT_MED_STATUS_C_U3M_BUREAU_BL', 'QT_MAX_STATUS_C_U3M_BUREAU_BL', 'QT_MIN_STATUS_C_U3M_BUREAU_BL', 'QT_TT_STATUS_3_U3M_BUREAU_BL', 'QT_MED_STATUS_3_U3M_BUREAU_BL', 'QT_MAX_STATUS_3_U3M_BUREAU_BL', 'QT_MIN_STATUS_3_U3M_BUREAU_BL', 'QT_TT_STATUS_0_U3M_BUREAU_BL', 'QT_MED_STATUS_0_U3M_BUREAU_BL', 'QT_MAX_STATUS_0_U3M_BUREAU_BL', 'QT_MIN_STATUS_0_U3M_BUREAU_BL', 'QT_TT_STATUS_2_U3M_BUREAU_BL', 'QT_MED_STATUS_2_U3M_BUREAU_BL', 'QT_MAX_STATUS_2_U3M_BUREAU_BL', 'QT_MIN_STATUS_2_U3M_BUREAU_BL', 'QT_TT_STATUS_X_U3M_BUREAU_BL', 'QT_MED_STATUS_X_U3M_BUREAU_BL', 'QT_MAX_STATUS_X_U3M_BUREAU_BL', 'QT_MIN_STATUS_X_U3M_BUREAU_BL', 'QT_TT_STATUS_5_U3M_BUREAU_BL', 'QT_MED_STATUS_5_U3M_BUREAU_BL', 'QT_MAX_STATUS_5_U3M_BUREAU_BL', 'QT_MIN_STATUS_5_U3M_BUREAU_BL', 'QT_TT_STATUS_1_U3M_BUREAU_BL', 'QT_MED_STATUS_1_U3M_BUREAU_BL', 'QT_MAX_STATUS_1_U3M_BUREAU_BL', 'QT_MIN_STATUS_1_U3M_BUREAU_BL', 'QT_TT_STA

In [0]:
from pyspark.sql.functions import current_date, date_format

# Adicionando as colunas de data ao DataFrame.
df_temp_04 = df_temp_03.withColumn('PK_DATREF', date_format(current_date(), 'yyyyMMdd')) \
                       .withColumn('PK_DAT_PROC', current_date())


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_04.columns
nomes_cols_novas = nomes_cols[1:-2]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_04.columns) - 3)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_04.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_04.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

# Mostrando o novo DataFrame com as variáveis criadas.
df_temp_04.show(5, truncate=False)

Quantidade Total de Variáveis Criadas: 112
Nomes das Variáveis Criadas: ['QT_TT_STATUS_C_U3M_BUREAU_BL', 'QT_MED_STATUS_C_U3M_BUREAU_BL', 'QT_MAX_STATUS_C_U3M_BUREAU_BL', 'QT_MIN_STATUS_C_U3M_BUREAU_BL', 'QT_TT_STATUS_3_U3M_BUREAU_BL', 'QT_MED_STATUS_3_U3M_BUREAU_BL', 'QT_MAX_STATUS_3_U3M_BUREAU_BL', 'QT_MIN_STATUS_3_U3M_BUREAU_BL', 'QT_TT_STATUS_0_U3M_BUREAU_BL', 'QT_MED_STATUS_0_U3M_BUREAU_BL', 'QT_MAX_STATUS_0_U3M_BUREAU_BL', 'QT_MIN_STATUS_0_U3M_BUREAU_BL', 'QT_TT_STATUS_2_U3M_BUREAU_BL', 'QT_MED_STATUS_2_U3M_BUREAU_BL', 'QT_MAX_STATUS_2_U3M_BUREAU_BL', 'QT_MIN_STATUS_2_U3M_BUREAU_BL', 'QT_TT_STATUS_X_U3M_BUREAU_BL', 'QT_MED_STATUS_X_U3M_BUREAU_BL', 'QT_MAX_STATUS_X_U3M_BUREAU_BL', 'QT_MIN_STATUS_X_U3M_BUREAU_BL', 'QT_TT_STATUS_5_U3M_BUREAU_BL', 'QT_MED_STATUS_5_U3M_BUREAU_BL', 'QT_MAX_STATUS_5_U3M_BUREAU_BL', 'QT_MIN_STATUS_5_U3M_BUREAU_BL', 'QT_TT_STATUS_1_U3M_BUREAU_BL', 'QT_MED_STATUS_1_U3M_BUREAU_BL', 'QT_MAX_STATUS_1_U3M_BUREAU_BL', 'QT_MIN_STATUS_1_U3M_BUREAU_BL', 'QT_TT_STA

## Salvando os dados em parquet

In [0]:
# Reparticionar para um único arquivo
df_temp_05 = df_temp_04.repartition(1)


df_temp_05.write.mode("overwrite").partitionBy("PK_DATREF").parquet('/FileStore/tables/podbank/bureau_balance')